In [18]:
import numpy as np
import pickle as pk
import pandas as pd
import timeit as tm
import csv
import sys

# Opening data

In [19]:
train_dat = pd.read_csv('../data/clean_data/train_vectors.csv', index_col=0, encoding='utf-8')
del train_dat['TYPE']
del train_dat['YEAR']
print train_dat

        MONTH   LATITUDE   LONGITUDE  N_VICTORIA-FRASERVIEW  N_WEST_END  \
148134      5  49.283335 -123.112899                      0           0   
148135      5  49.283335 -123.112899                      0           0   
148136      6  49.283335 -123.112899                      0           0   
148137      6  49.283335 -123.112899                      0           0   
148138      6  49.283335 -123.112899                      0           0   
148139      6  49.283335 -123.112899                      0           0   
148140      6  49.283335 -123.112899                      0           0   
148141      6  49.283335 -123.112899                      0           0   
148142      6  49.283335 -123.112899                      0           0   
148143      7  49.283335 -123.112899                      0           0   
148144      8  49.283335 -123.112899                      0           0   
148145      8  49.283335 -123.112899                      0           0   
148146      8  49.283335 

In [20]:
train_lbl = pd.read_csv('../data/clean_data/train_labels.csv', index_col=0, encoding='utf-8')
print train_lbl

        BNE Commercial  BNE Residential/Other  Mischief  Other Theft  \
148134               0                      0         0            0   
148135               0                      0         0            0   
148136               0                      0         0            0   
148137               0                      0         0            0   
148138               0                      0         0            0   
148139               0                      0         0            0   
148140               0                      0         0            0   
148141               0                      0         0            0   
148142               0                      0         0            0   
148143               0                      0         0            0   
148144               0                      0         0            0   
148145               0                      0         0            0   
148146               0                      0         0         

In [21]:
headers = [list(train_dat)]
dat_mtx = train_dat.as_matrix()
print dat_mtx.shape
lbl_mtx = train_lbl.as_matrix()
print lbl_mtx.shape

(295169L, 63L)
(295169L, 7L)


In [22]:
# Open test data
test_dat = pd.read_csv('../data/clean_data/test_vectors.csv', index_col=0, encoding='utf-8')
del test_dat['TYPE']
del test_dat['YEAR']
print test_dat

        MONTH   LATITUDE   LONGITUDE  N_VICTORIA-FRASERVIEW  N_WEST_END  \
444989      2  49.271601 -123.126966                      0           0   
444990      1  49.282477 -123.118136                      0           0   
444991      8  49.282477 -123.118136                      0           0   
444992      9  49.282477 -123.118136                      0           0   
444993      6  49.253906 -123.112624                      0           0   
444994      5  49.223887 -123.129722                      0           0   
444995      3  49.225733 -123.128743                      0           0   
444996     12  49.266030 -123.130657                      0           0   
444997     12  49.208469 -123.132941                      0           0   
444998      3  49.250341 -123.072630                      0           0   
444999      3  49.250107 -123.101081                      0           0   
445000      6  49.236121 -123.193651                      0           0   
445001      8  49.287723 

In [23]:
test_lbl = pd.read_csv('../data/clean_data/test_labels.csv', index_col=0, encoding='utf-8')
print test_lbl

        BNE Commercial  BNE Residential/Other  Mischief  Other Theft  \
444989               0                      0         0            0   
444990               0                      0         1            0   
444991               0                      0         1            0   
444992               0                      0         1            0   
444993               0                      0         0            0   
444994               0                      1         0            0   
444995               0                      1         0            0   
444996               0                      0         0            0   
444997               0                      0         0            0   
444998               0                      1         0            0   
444999               0                      0         0            1   
445000               0                      0         0            0   
445001               0                      0         0         

In [24]:
dat_mtx_tst = test_dat.as_matrix()
print dat_mtx_tst.shape
lbl_mtx_tst = test_lbl.as_matrix()
print lbl_mtx_tst.shape

(34142L, 63L)
(34142L, 7L)


In [30]:
from sklearn.preprocessing import StandardScaler
# Scale data
scaler = StandardScaler()
scaler.fit(dat_mtx)

scaled_dat_mtx = scaler.transform(dat_mtx, copy=True)
scaled_dat_mtx_tst = scaler.transform(dat_mtx_tst, copy=True)

# Linear Regression

In [25]:
from sklearn.linear_model import LinearRegression

# Fit Linear Regression
lin_reg = LinearRegression(n_jobs=-1)
lin_reg.fit(dat_mtx, lbl_mtx)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [26]:
# Generate predictions
predictions = lin_reg.predict(dat_mtx_tst)
print predictions.shape

(34142L, 7L)


In [27]:
import math

errors = []

# compute squared errors
for i in xrange(predictions.shape[0]):
    p = predictions[i]
    t = lbl_mtx_tst[i]
    
    # compute distance
    squared_distance = 0.0
    for j in xrange(predictions.shape[1]):
        squared_distance += (p[j] - t[j])**2
    
    errors.append(squared_distance)

rmse = math.sqrt(sum(errors)/len(errors))
print 'Root mean squared error:', rmse

Root mean squared error: 5.29110079844


# Scaled Linear Regression

In [32]:
import math
from sklearn.linear_model import LinearRegression

# Fit Linear Regression
lin_reg = LinearRegression(n_jobs=-1)
lin_reg.fit(scaled_dat_mtx, lbl_mtx)

# Generate predictions
predictions = lin_reg.predict(scaled_dat_mtx_tst)
print predictions.shape

errors = []

# compute squared errors
for i in xrange(predictions.shape[0]):
    p = predictions[i]
    t = lbl_mtx_tst[i]
    
    # compute distance
    squared_distance = 0.0
    for j in xrange(predictions.shape[1]):
        squared_distance += (p[j] - t[j])**2
    
    errors.append(squared_distance)

rmse = math.sqrt(sum(errors)/len(errors))
print 'Root mean squared error:', rmse

(34142L, 7L)
Root mean squared error: 5.29202255033


Interestingly Scaled Linear Regression did not improve results.

In [36]:
from sklearn.linear_model import BayesianRidge

clf = BayesianRidge()
clf.fit(dat_mtx, lbl_mtx)

ValueError: bad input shape (295169L, 7L)